In [ ]:
import numpy as np
import os
os.environ['KERAS_BACKEND']        = 'tensorflow'
os.environ['CUDA_DEVICE_ORDER']    = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config = config)

import keras.backend as K
K.set_image_data_format('channels_last')

In [ ]:
import keras
from keras.models import Model
from keras.layers import Conv2D, Activation, Dropout, Flatten, Dense, Input, MaxPooling2D, AveragePooling2D, BatchNormalization
from keras.initializers import RandomNormal
from keras.optimizers import Adam
from keras.layers.merge import Concatenate, add
from keras.utils import multi_gpu_model

In [ ]:
_conv_init = RandomNormal(0, 0.02)
_gamma_init = RandomNormal(1., 0.02)

In [ ]:
def CONV(f, *a, **k):
    return Conv2D(f, kernel_initializer = _conv_init, *a, **k)

def NORM():
    return BatchNormalization(momentum=0.9, axis=-1, epsilon=1.01e-5, gamma_initializer = _gamma_init)

def POOL(*a, **k):
    return MaxPooling2D(*a, **k)

def BLOCK(_input, _nb_feature):
    _nb_inception = _nb_feature // 4
    
    _layer_1 = CONV(_nb_inception, (1, 1), strides = 1, padding = 'same') (_input)
    _layer_1 = NORM() (_layer_1, training = 1)
    _layer_1 = Activation('relu') (_layer_1)

    _layer_2 = CONV(_nb_inception, (1, 3), strides = 1, padding = 'same') (_input)
    _layer_2 = NORM() (_layer_2, training = 1)
    _layer_2 = Activation('relu') (_layer_2)

    _layer_3 = CONV(_nb_inception, (3, 1), strides = 1, padding = 'same') (_input)
    _layer_3 = NORM() (_layer_3, training = 1)
    _layer_3 = Activation('relu') (_layer_3)

    _layer_4 = CONV(_nb_inception, (3, 3), strides = 1, padding = 'same') (_input)
    _layer_4 = NORM() (_layer_4, training = 1)
    _layer_4 = Activation('relu') (_layer_4)
    
    _layer = Concatenate(-1)([_layer_1, _layer_2, _layer_3, _layer_4])
    _layer = CONV(_nb_inception * 2, (1, 1), strides = 1, padding = 'same') (_layer)
    _layer = Activation('relu') (_layer)
    _layer = CONV(_nb_feature, (3, 3), strides = 1, padding = 'same') (_layer)
    _layer = Activation('relu') (_layer)
    
    return _layer

def CoFFee(_isize, _nc_in, _nc_out, _growth_rate = 64, _nb_block = 3):

    _input = Input(shape = (_isize, _isize, _nc_in))

    _nb_feature = 64
    _layer = CONV(_nb_feature, (3, 3), strides = 1, padding = 'same') (_input)
    _layer = Activation('relu') (_layer)
    
    for _i in range(_nb_block):
        
        _ip = _layer
        _nb_feature += _growth_rate
        _layer = BLOCK(_ip, _nb_feature)

        _ip = Concatenate(-1) ([_ip, _layer])
        _nb_feature += _growth_rate
        _layer = BLOCK(_ip, _nb_feature)

        _ip = Concatenate(-1) ([_ip, _layer])
        _nb_feature += _growth_rate
        _layer = BLOCK(_ip, _nb_feature)
        
        _layer = POOL(pool_size = (3, 3), strides = 2, padding = 'same') (_layer)
        
    _shape = _layer.get_shape().as_list()
    
    _layer = AveragePooling2D(pool_size = (_shape[1], _shape[2]), strides = (_shape[1], _shape[2]), padding = 'same') (_layer)

    _layer = Flatten() (_layer)
    
    _layer = Dense(units = _nc_out) (_layer)
    _layer = ACtivation('softmax') (_layer)
    
    return Model(inputs = _input, outputs = _layer)

In [ ]:
_isize = 256
_nb_block = 3
_nc_in = 1
_nc_out = 2
_bsize = 4
_growth_rate = 64
_max_epoch = 100

In [ ]:
_model = CoFFee(_isize, _nc_in, _nc_out, _growth_rate, _nb_block)
_model.summary()

In [ ]:
def weighted_categorical_crossentropy(weights):
    weights = K.variable(weights)
    def loss(y_true, y_pred):
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    return loss

from keras.optimizers import Adam
_adam = Adam(lr = 2e-4)
_weights = np.array([1.0,1.0])
_model.compile(loss=weighted_categorical_crossentropy(_weights), optimizer=_adam, metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

_path_training = ''
_path_test = ''
_color_mode = 'grayscale'

_generator_training = ImageDataGenerator(rescale = 1./255., validation_split = 0.1)
_dataset_training = _generator_training.flow_from_directory(_path_train,
                                                            target_size = (_isize, _isize),
                                                            batch_size = _bsize,
                                                            class_mode = 'categorical',
                                                            color_mode = _color_mode,
                                                            subset = 'training')

_dataset_validation = _generator_training.flow_from_directory(_path_grain,
                                                              target_size = (_isize, _isize),
                                                              batch_size = _bsize,
                                                              class_mode = 'categorical',
                                                              color_mode = _color_mode,
                                                              subset = 'validation')

_generator_testing = ImageDataGenerator(rescale = 1./255.)
_dataset_testing = _generator_testing.flow_from_directory(_path_test,
                                                          target_size = (_isize, _isize),
                                                          batch_size = _bsize,
                                                          class_mode = 'categorical',
                                                          color_mode = _color_mode,
                                                          shuffle = False)



_nb_batch_training = int(_dataset_training.n / _bsize)
_nb_batch_validation = int(_dataset_validation.n / _bsize)
_nb_batch_testing = int(_dataset_testing.n / _bsize)

In [ ]:
class CUSTOM_HISTORY(keras.callbacks.Callback):
    def init(self):
        self.train_loss = []
        self.val_loss = []
        self.train_acc = []
        self.val_acc = []
    def on_epoch_end(self, batch, logs = {}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
        self.train_acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
CUSTOM_CALLBACK = CUSTOM_HISTORY()
CUSTOM_CALLBACK.init()

def STATISTICS(_labels, _results):
    _h, _f, _m, _n = 0, 0, 0, 0

    assert len(_labels) == (_results.shape[0])
    
    for _i in range(len(_labels)):
        
        _label = _labels[_i]
        _result = np.argmax(_results[_i])
        
        if _label == 1 and _result == 1 :
            _h += 1
        elif _label == 1 and _result == 0 :
            _m += 1
        elif _label == 0 and _result == 1 :
            _f += 1
        elif _label == 0 and _result == 0 :
            _n += 1
        else :
            pass
        
    _acc = (_h + _n)/(_h + _m + _f + _n)
    _pod = _h/(_h + _m)
    _far = _f/(_h + _f)
    _csi = _h/(_h + _m + _f)
    _hss = (2 * (_h * _n - _f * _m)) / ((_h + _m) * (_m + _n) + (_h + _f) * (_f + _n))
    _tss = (_h * N - _f * _m)/ ((_h + _m) * (_f + _n))
    
    return _acc, _pod, _csi, _far, _hss, _tss


_epoch = 1
while _epoch <= _max_epoch :
    
    _model.fit_generator(_dataset_training,
                         steps_per_epoch = _nb_batch_training,
                         validation_data = _dataset_validation,
                         epochs = 1,
                         callbacks = [CUSTOM_CALLBACK])
    
    _classes = _dataset_testing.classes
    _results = _model.predict_generator(_dataset_testing)
    
    _acc, _pod, _csi, _far, _hss, _tss = STATISTICS(_classes, _results)
    _showing = '%5d'%_epoch + '%7.4f'%_acc + '%7.4f'%_pod + '%7.4f'%_csi + '%7.4f'%_far + '%7.4f'%_hss + '%7.4f'%_tss
    print(_showing)
    
    _epoch += 1

In [ ]:
import matplotlib.pyplot as plt

FIG, LOSS_AX = plt.subplots()
ACC_AX = LOSS_AX.twinx()
LOSS_AX.plot(CUSTOM_HIST.train_loss, 'y', label = 'TRAIN LOSS')
LOSS_AX.plot(CUSTOM_HIST.val_loss, 'r', label = 'VAL LOSS')
ACC_AX.plot(CUSTOM_HIST.train_acc, 'b', label = 'TRAIN ACC')
ACC_AX.plot(CUSTOM_HIST.val_acc, 'g', label = 'VAL ACC')
LOSS_AX.set_xlabel('EPOCH')
LOSS_AX.set_ylabel('LOSS')
ACC_AX.set_ylabel('ACCURACY')
LOSS_AX.legend(loc='upper left')
ACC_AX.legend(loc='lower left')
#FIG.savefig('./' + NAME_MODEL + '.png')